In [ ]:
!which python

### Imports

In [ ]:
from turtle import distance
import matplotlib.pyplot as plt # plotting library
import numpy as np # this module is useful to work with numerical arrays
import pandas as pd 
import random 
import math
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader,random_split
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
import cv2
from torch.autograd import Variable
from torchvision.utils import save_image

In [1]:
import os

In [2]:
data_dir = '/media/data_cifs/anagara8/video_processed_frames/'

img_list = os.listdir(data_dir)

In [3]:
len(img_list)

618210

### WandB

In [ ]:
# Tracking training
import wandb

# wandb.init(project="nih_animal_behavior", entity="serrelab")

### Processing the Dataset

In [ ]:
data_dir = 'dataset'
number_of_frames_per_video = 2
real_train_data = []
dimension = (256, 256)

train_dataset_list = pickle.load(open('/home/anagara8/Documents/Autoencoder/postext_frames.pickle', 'rb'))


for data in train_dataset_list:
    temp = []
    if len(data) == number_of_frames_per_video:
        for frame_index in range(number_of_frames_per_video):
            temp.append(cv2.resize(cv2.cvtColor(data[frame_index], cv2.COLOR_BGR2GRAY), dimension, interpolation = cv2.INTER_AREA))
    
    else:
        continue
    real_train_data += temp
    
# Same data for training and testing
train_dataset_list = real_train_data
test_dataset_list = real_train_data

#### Converting to Tensors

In [ ]:
training_data_tensor = torch.Tensor(train_dataset_list)
testing_data_tensor = torch.Tensor(test_dataset_list)
print("Converted to Tensors")

#### Converting to Tensor Dataset

In [ ]:
training_dataset = TensorDataset(training_data_tensor)
testing_dataset = TensorDataset(testing_data_tensor)
print("Converted to Tensor Dataset")

#### Print a sample of the dataset

In [ ]:
print(len(training_dataset), "x", len(training_dataset[0]), "x", len(training_dataset[0][0]), "x", len(training_dataset[0][0][0]), "x", 1)
for i in training_dataset:
    print(i)
    break

#### Train-Validation Split

In [ ]:
m=len(training_dataset)

print("Length:",m, "-> Training set size:", int(math.ceil(m-m*0.2)), "| Validation set size:", int(m*0.2))
train_data, val_data = random_split(training_dataset, [int(math.ceil(m-m*0.2)), int(m*0.2)])

#### Converting to DataLoader

In [ ]:
batch_size=256
train_set_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
valid_set_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
test_set_loader = torch.utils.data.DataLoader(testing_dataset, batch_size=batch_size,shuffle=True)
print("Converted to DataLoader")

#### Looking at one example

In [ ]:
examples = enumerate(train_set_loader.dataset)
batch_idx, example_data = next(examples)
batch_idx, example_data

In [ ]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(256 * 256, 256),
            nn.ReLU(True),
            nn.Linear(256, 128),
            nn.ReLU(True), nn.Linear(128, 64), nn.ReLU(True), nn.Linear(64, 32))
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Linear(128, 256),
            nn.ReLU(True), nn.Linear(256, 256 * 256), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-02, weight_decay=1e-5)

In [ ]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 256, 256)
    return x

In [ ]:
num_epochs = 1
for epoch in range(num_epochs):
    for data in train_set_loader:
        img = data[0]
#         print(img)
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './sim_autoencoder.pth')

In [ ]:
model = Autoencoder()
max_epochs = 20
outputs = train(model, num_epochs=max_epochs)